In [1]:
using FermiCG
using PyCall
using Plots
using LinearAlgebra
using Printf

## Import PYSCF data

In [2]:
pyscf = pyimport("pyscf");
fcidump = pyimport("pyscf.tools.fcidump");
ctx = fcidump.read("fe2s2");
h = ctx["H1"];
g = ctx["H2"];
ecore = ctx["ECORE"];
g = pyscf.ao2mo.restore("1", g, size(h,2));

/Users/nmayhall/code/FermiCG/src/python/venv/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [3]:
# blocking from paper
blocks = [[0,1],[2,3,4,5,6],[7,9,11],[8,10,12],[13,14,15,16,17],[18,19]]
init_fspace = [(2,2),(5,0),(3,3),(3,3),(5,0),(2,2)]

for b in blocks
    b .+= 1
end
clusters = [Cluster(i,collect(blocks[i])) for i = 1:length(blocks)]
display(clusters)

6-element Array{Cluster,1}:
 Cluster(1, [1, 2])
 Cluster(2, [3, 4, 5, 6, 7])
 Cluster(3, [8, 10, 12])
 Cluster(4, [9, 11, 13])
 Cluster(5, [14, 15, 16, 17, 18])
 Cluster(6, [19, 20])

In [ ]:
# blocking 1
blocks = [  [0,1,2,3,6,7,8],        # L1(sig)+ Fe1(t) + L2(sig)
            [11,12,13,16,17,18,19], # L2(sig)+ Fe2(t) + L3(sig)
            [4,5,9,10,14,15],       # Fe1(e) + Fe2(e) + L2(pi)
         ]

init_fspace = ((7,4), (7,4), (4,4))

for b in blocks
    b .+= 1
end
clusters = [Cluster(i,collect(blocks[i])) for i = 1:length(blocks)]
display(clusters)


In [ ]:
# blocking 2
blocks = [  [0,1],                     # L1(sig)
            [2,3,4,5,6,13,14,15,16,17],# Fe1 Fe2
            [7,8,9,10,11,12],          # L2(sig, pi)
            [18,19],                   # L3(sig)
         ]

init_fspace = [(2,2), (5,5), (6,6), (2,2)]

for b in blocks
    b .+= 1
end
clusters = [Cluster(i,collect(blocks[i])) for i = 1:length(blocks)]
display(clusters)

In [4]:
ints = InCoreInts(ecore,h,g);

# U = FermiCG.block_diagonalize_orbitals(h,clusters);
# ints = FermiCG.orbital_rotation(ints,U);


na = 0
nb = 0
for f in init_fspace
    na += f[1]
    nb += f[2]
end
display(na)
display(nb)
# get initial guess for 1RDM
tmp = eigen(ints.h1).vectors;
rdm1a_guess = tmp[:,1:na]*tmp[:,1:na]';
rdm1b_guess = tmp[:,1:nb]*tmp[:,1:nb]';
# eigen(ints.h1).values


20

10

In [5]:
e_cmf, U, Da, Db  = FermiCG.cmf_oo(ints, clusters, init_fspace, rdm1a_guess, rdm1b_guess,
                                        max_iter_oo=40, verbose=0, gconv=1e-7, method="bfgs");
ints2 = FermiCG.orbital_rotation(ints,U);


 ooCMF Iter:    1 Total= -115.958971122269 Active= -115.958971122269 G=     5.68e-01
 ooCMF Iter:    2 Total= -116.452566166470 Active= -116.452566166470 G=     2.75e-01
 ooCMF Iter:    3 Total= -116.487390854933 Active= -116.487390854933 G=     1.43e-01
 ooCMF Iter:    4 Total= -116.503317395887 Active= -116.503317395887 G=     6.98e-02
 ooCMF Iter:    5 Total= -116.506555919580 Active= -116.506555919580 G=     2.48e-02
 ooCMF Iter:    6 Total= -116.506990651447 Active= -116.506990651447 G=     8.78e-03
 ooCMF Iter:    7 Total= -116.507077222629 Active= -116.507077222629 G=     6.40e-03
 ooCMF Iter:    8 Total= -116.507125519400 Active= -116.507125519400 G=     2.62e-03
 ooCMF Iter:    9 Total= -116.507130990476 Active= -116.507130990476 G=     1.05e-03
 ooCMF Iter:   10 Total= -116.507131947301 Active= -116.507131947301 G=     5.05e-04
 ooCMF Iter:   11 Total= -116.507132184537 Active= -116.507132184537 G=     2.42e-04
 ooCMF Iter:   12 Total= -116.507132234567 Active= -116.507132234

 * Status: success

 * Candidate solution
    Final objective value:     -1.165071e+02

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 4.87e-07 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.27e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 3.27e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.81e-15 ≰ 0.0e+00
    |g(x)|                 = 5.69e-08 ≤ 1.0e-07

 * Work counters
    Seconds run:   2  (vs limit Inf)
    Iterations:    19
    f(x) calls:    57
    ∇f(x) calls:   57


 ooCMF Iter:   20 Total= -116.507132250720 Active= -116.507132250720 G=     2.38e-07
*ooCMF -116.50713225 


# Compute `cluster_bases` and `cluster_ops`


In [6]:
max_roots = 300

# Average the alpha/beta densities to avoid spin-contamination in basis
Da = 0.5*(Da.+Db)
Da = Db

#
# Build Cluster basis
cluster_bases = FermiCG.compute_cluster_eigenbasis(ints2, clusters, verbose=0, max_roots=max_roots,
        init_fspace=init_fspace, rdm1a=Da, rdm1b=Db);

In [7]:
#
# Build ClusteredOperator
clustered_ham = FermiCG.extract_ClusteredTerms(ints2, clusters);


# Build Cluster Operators
cluster_ops = FermiCG.compute_cluster_ops(cluster_bases, ints2);

# Add cmf hamiltonians for doing MP-style PT2 
FermiCG.add_cmf_operators!(cluster_ops, cluster_bases, ints2, Da, Db, verbose=0);

 Remove duplicates
 Number of terms reduced from  5226 to  2226


In [8]:
nroots = 6
ci_vector = FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(5,0),(3,3),(3,3),(0,5),(2,2)]), R=nroots);
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(4,1),(3,3),(3,3),(1,4),(2,2)]),R=nroots))
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(3,2),(3,3),(3,3),(2,3),(2,2)]),R=nroots))
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(2,3),(3,3),(3,3),(3,2),(2,2)]),R=nroots))
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(1,4),(3,3),(3,3),(4,1),(2,2)]),R=nroots))
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(0,5),(3,3),(3,3),(5,0),(2,2)]),R=nroots))


for (fock,configs) in deepcopy(ci_vector.data)
    for ci in clusters
        for cj in clusters
            #a
            new_fock = [fock...]
            new_fock[ci.idx] = (new_fock[ci.idx][1]+1, new_fock[ci.idx][2])
            new_fock[cj.idx] = (new_fock[cj.idx][1]-1, new_fock[cj.idx][2])
            new_fock = FermiCG.FockConfig(new_fock)
            all([f[1]>=0 && f[1]<=length(clusters[ii]) for (ii,f) in enumerate(new_fock)]) || continue
#             FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, new_fock,R=nroots))
        end
    end
    
    for ci in clusters
        for cj in clusters
            #b
            new_fock = [fock...]
            new_fock[ci.idx] = (new_fock[ci.idx][1], new_fock[ci.idx][2]+1)
            new_fock[cj.idx] = (new_fock[cj.idx][1], new_fock[cj.idx][2]-1)
            new_fock = FermiCG.FockConfig(new_fock)
            
            all([f[2]>=0 && f[2]<=length(clusters[ii]) for (ii,f) in enumerate(new_fock)]) || continue
#             FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, new_fock,R=nroots))
        end
    end
end
        
display(ci_vector)


 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =     6  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 --------------------------------------------------


In [9]:
e0, v0 = FermiCG.tpsci_ci(ci_vector, cluster_ops, clustered_ham,
                            thresh_cipsi=5e-3, # Threshold for adding to P-space
                            thresh_foi=1e-5,    # Threshold for keeping terms when defining FOIS    
                            thresh_asci=-1,     # Threshold of P-space configs to search from
                            max_iter=10,
                            do_s2=true,
                            matvec=3);


 ci_vector     :(6, 6)
 thresh_cipsi  :0.005
 thresh_foi    :1.0e-5
 thresh_asci   :-1
 max_iter      :10
 conv_thresh   :0.0001

     Selected CI Iteration:    1 epsilon:   0.00500000
 In open_matvec_thread2

 Number of jobs:    6
 Number of threads: 12
  1.573506 seconds (2.92 M allocations: 138.428 MiB)
 Now collect thread results
 In open_matvec_thread2

 Number of jobs:    6
 Number of threads: 12
  0.004043 seconds (2.07 k allocations: 102.531 KiB)
 Now collect thread results
 Add spin purifying configs to current space      6 →      6
 Build Hamiltonian matrix with dimension:     6
  0.417006 seconds (954.51 k allocations: 48.390 MiB)
  Root       Energy           S2
     1 -116.50713225  30.00000000
     2 -116.49935728  20.00000000
     3 -116.49313731  12.00000000
     4 -116.48847232   6.00000000
     5 -116.48536234   2.00000000
     6 -116.48380734   0.00000000
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =     6  
 ------

  0.007322 seconds (40.21 k allocations: 1.419 MiB)
 Now collect thread results
 In open_matvec_thread2

 Number of jobs:    16
 Number of threads: 12
  0.004979 seconds (42.01 k allocations: 1.440 MiB)
 Now collect thread results
 Add spin purifying configs to current space    168 →    176
 Build Hamiltonian matrix with dimension:   176
  0.181108 seconds (1.42 M allocations: 59.974 MiB)
  Root       Energy           S2
     1 -116.50919419   0.00000000
     2 -116.50917852   2.00000000
     3 -116.50910415   6.00000000
     4 -116.50887583  12.00000000
     5 -116.50832168  20.00000000
     6 -116.50713225  30.00000000
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =   176  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ---------

 Number of jobs:    16
 Number of threads: 12
  0.007192 seconds (42.01 k allocations: 1.440 MiB)
 Now collect thread results
 In open_matvec_thread2

 Number of jobs:    16
 Number of threads: 12
  0.005393 seconds (42.01 k allocations: 1.440 MiB)
 Now collect thread results
 Add spin purifying configs to current space    176 →    176
 Length of ASCI vector      176 →      176 
 Norm of delta v:
   1.41421356
   1.41421356
   1.41421356
   1.41421356
   1.41421356
   1.41421356
 In open_matvec_thread2

 Number of jobs:    886
 Number of threads: 12
  1.552693 seconds (2.39 M allocations: 173.172 MiB)
 Now collect thread results
  1.573579 seconds (2.60 M allocations: 204.377 MiB)
 Length of FOIS vector: 448550
 Length of FOIS vector: 448374
 Compute diagonal
  0.512385 seconds (11.21 M allocations: 174.852 MiB)

  Root         E(0)         E(2)
     1 -116.50919419 -116.52482432
     2 -116.50917852 -116.52389298
     3 -116.50910415 -116.52261851
     4 -116.50887583 -116.52063497
  

 0.134               1                   ( 2,2 )( 0,5 )( 3,3 )( 3,3 )( 5,0 )( 2,2 )
 0.006               11                  ( 2,2 )( 2,4 )( 3,3 )( 3,3 )( 3,1 )( 2,2 )
 0.006               11                  ( 2,2 )( 4,2 )( 3,3 )( 3,3 )( 1,3 )( 2,2 )
 0.006               11                  ( 2,2 )( 1,3 )( 3,3 )( 3,3 )( 4,2 )( 2,2 )
 0.002               11                  ( 2,2 )( 1,5 )( 3,3 )( 3,3 )( 4,0 )( 2,2 )
 0.002               11                  ( 2,2 )( 0,4 )( 3,3 )( 3,3 )( 5,1 )( 2,2 )
 0.006               11                  ( 2,2 )( 3,1 )( 3,3 )( 3,3 )( 2,4 )( 2,2 )
 0.002               11                  ( 2,2 )( 5,1 )( 3,3 )( 3,3 )( 0,4 )( 2,2 )
 0.002               11                  ( 2,2 )( 4,0 )( 3,3 )( 3,3 )( 1,5 )( 2,2 )
 --------------------------------------------------
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =   483  
 ----------                root ------:     =     5  
 -------------------------------

 0.087               105                 ( 2,2 )( 3,2 )( 3,3 )( 3,3 )( 2,3 )( 2,2 )
 0.087               105                 ( 2,2 )( 2,3 )( 3,3 )( 3,3 )( 3,2 )( 2,2 )
 0.263               105                 ( 2,2 )( 1,4 )( 3,3 )( 3,3 )( 4,1 )( 2,2 )
 0.134               1                   ( 2,2 )( 0,5 )( 3,3 )( 3,3 )( 5,0 )( 2,2 )
 0.006               11                  ( 2,2 )( 2,4 )( 3,3 )( 3,3 )( 3,1 )( 2,2 )
 0.006               11                  ( 2,2 )( 4,2 )( 3,3 )( 3,3 )( 1,3 )( 2,2 )
 0.006               11                  ( 2,2 )( 1,3 )( 3,3 )( 3,3 )( 4,2 )( 2,2 )
 0.002               11                  ( 2,2 )( 1,5 )( 3,3 )( 3,3 )( 4,0 )( 2,2 )
 0.002               11                  ( 2,2 )( 0,4 )( 3,3 )( 3,3 )( 5,1 )( 2,2 )
 0.006               11                  ( 2,2 )( 3,1 )( 3,3 )( 3,3 )( 2,4 )( 2,2 )
 0.002               11                  ( 2,2 )( 5,1 )( 3,3 )( 3,3 )( 0,4 )( 2,2 )
 0.002               11                  ( 2,2 )( 4,0 )( 3,3 )( 3,3 )( 1,5 )

In [ ]:
vec0 = FermiCG.get_vectors(v0)[:,1]

In [ ]:
ci_vector = FermiCG.ClusteredState(clusters, FermiCG.FockConfig(init_fspace), R=1);
for (fock,configs) in v0.data
    FermiCG.add_fockconfig!(ci_vector,fock)
end

for (fock,configs) in v0.data
    for (config,coeffs) in configs
         ci_vector[fock][config] = [coeffs[1]]
    end
end
FermiCG.clip!(ci_vector,thresh=1e-3)
display(ci_vector)

In [10]:
e1, v1 = FermiCG.tpsci_ci(v0, cluster_ops, clustered_ham,
                            thresh_cipsi=1e-3, # Threshold for adding to P-space
                            thresh_foi=1e-5,    # Threshold for keeping terms when defining FOIS    
                            thresh_asci=-1,     # Threshold of P-space configs to search from
                            max_iter=10,
                            do_s2=true,
                            incremental=true,
                            matvec=3);

 ci_vector     :(532, 6)
 thresh_cipsi  :0.001
 thresh_foi    :1.0e-5
 thresh_asci   :-1
 max_iter      :10
 conv_thresh   :0.0001

     Selected CI Iteration:    1 epsilon:   0.00100000
 In open_matvec_thread2

 Number of jobs:    16
 Number of threads: 12
  0.014360 seconds (123.62 k allocations: 4.039 MiB)
 Now collect thread results
 In open_matvec_thread2

 Number of jobs:    16
 Number of threads: 12
  0.016372 seconds (123.63 k allocations: 4.040 MiB)
 Now collect thread results
 Add spin purifying configs to current space    532 →    532
 Build Hamiltonian matrix with dimension:   532
  0.117170 seconds (3.72 M allocations: 89.271 MiB)
  Root       Energy           S2
     1 -116.51088649   0.00000000
     2 -116.51064398   2.00000000
     3 -116.51015349   6.00000000
     4 -116.50940640  12.00000000
     5 -116.50839669  20.00000000
     6 -116.50713225  30.00000000
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =   532  
 ----

  0.015920 seconds (124.28 k allocations: 4.108 MiB)
 Now collect thread results
 In open_matvec_thread2

 Number of jobs:    16
 Number of threads: 12
  0.015508 seconds (124.28 k allocations: 4.108 MiB)
 Now collect thread results
 Add spin purifying configs to current space    532 →    532
 Length of ASCI vector      532 →      532 
 Norm of delta v:
   1.00000000
   1.00000000
   1.00000000
   1.00000000
   1.00000000
   1.00000000
 In open_matvec_thread2

 Number of jobs:    886
 Number of threads: 12
  5.283506 seconds (6.23 M allocations: 434.020 MiB)
 Now collect thread results
  5.308350 seconds (6.45 M allocations: 465.757 MiB)
 Length of FOIS vector: 710994
 Length of FOIS vector: 710462
 Compute diagonal
  0.807903 seconds (17.76 M allocations: 277.063 MiB)

  Root         E(0)         E(2)
     1 -116.51088649 -116.52535928
     2 -116.51064398 -116.52420983
     3 -116.51015349 -116.52285987
     4 -116.50940640 -116.52067392
     5 -116.50839669 -116.51807867
     6 -116

 0.371               396                 ( 2,2 )( 2,3 )( 3,3 )( 3,3 )( 3,2 )( 2,2 )
 0.093               396                 ( 2,2 )( 1,4 )( 3,3 )( 3,3 )( 4,1 )( 2,2 )
 0.004               1                   ( 2,2 )( 0,5 )( 3,3 )( 3,3 )( 5,0 )( 2,2 )
 0.001               11                  ( 2,2 )( 3,4 )( 2,2 )( 3,3 )( 3,2 )( 2,2 )
 0.001               11                  ( 2,2 )( 4,3 )( 3,3 )( 2,2 )( 2,3 )( 2,2 )
 0.001               11                  ( 2,2 )( 3,4 )( 3,3 )( 2,2 )( 3,2 )( 2,2 )
 0.001               11                  ( 2,2 )( 2,3 )( 2,2 )( 3,3 )( 4,3 )( 2,2 )
 0.001               11                  ( 2,2 )( 3,2 )( 3,3 )( 2,2 )( 3,4 )( 2,2 )
 0.001               11                  ( 2,2 )( 2,3 )( 3,3 )( 2,2 )( 4,3 )( 2,2 )
 0.001               11                  ( 2,2 )( 3,2 )( 2,2 )( 3,3 )( 3,4 )( 2,2 )
 0.001               11                  ( 2,2 )( 4,3 )( 2,2 )( 3,3 )( 2,3 )( 2,2 )
 --------------------------------------------------
 In open_matvec_thread2


 0.002               7                   ( 2,2 )( 5,0 )( 2,3 )( 3,3 )( 1,5 )( 2,2 )
 0.001               11                  ( 2,2 )( 5,2 )( 3,3 )( 3,1 )( 0,5 )( 2,2 )
 0.001               11                  ( 2,2 )( 2,5 )( 1,3 )( 3,3 )( 5,0 )( 2,2 )
 0.001               14                  ( 2,2 )( 0,5 )( 3,3 )( 3,2 )( 5,2 )( 2,1 )
 0.001               11                  ( 2,2 )( 5,0 )( 1,3 )( 3,3 )( 2,5 )( 2,2 )
 --------------------------------------------------
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =  9368  
 ----------                root ------:     =     3  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.251               1                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 0,5 )( 2,2 )
 0.011               590                 ( 2,2 )( 4,1 )( 3,3 

 Number of jobs:    346
 Number of threads: 12
  0.286235 seconds (2.33 M allocations: 76.743 MiB)
 Now collect thread results
 In open_matvec_thread2

 Number of jobs:    346
 Number of threads: 12
  0.281459 seconds (2.33 M allocations: 76.821 MiB)
 Now collect thread results
 Add spin purifying configs to current space   9368 →   9382
 Length of ASCI vector     9382 →     9382 
 Norm of delta v:
   0.16494230
   0.16710106
   0.14771788
   0.14228760
   0.13624518
   0.12406152
 In open_matvec_thread2

 Number of jobs:    6716
 Number of threads: 12
 46.471097 seconds (58.61 M allocations: 2.211 GiB)
 Now collect thread results
 47.287231 seconds (63.26 M allocations: 2.413 GiB, 0.69% gc time)
 Length of FOIS vector: 2465157
 Length of FOIS vector: 2455775
 Compute diagonal
  3.064771 seconds (61.39 M allocations: 983.291 MiB, 11.69% gc time)

  Root         E(0)         E(2)
     1 -116.56484898 -116.57334743
     2 -116.56419339 -116.57163452
     3 -116.56288628 -116.56980754
   

...

 0.001               11                  ( 2,2 )( 5,0 )( 1,3 )( 3,3 )( 2,5 )( 2,2 )
 --------------------------------------------------
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim = 13988  
 ----------                root ------:     =     4  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.114               1                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 0,5 )( 2,2 )
 0.231               606                 ( 2,2 )( 4,1 )( 3,3 )( 3,3 )( 1,4 )( 2,2 )
 0.073               2314                ( 2,2 )( 3,2 )( 3,3 )( 3,3 )( 2,3 )( 2,2 )
 0.082               2314                ( 2,2 )( 2,3 )( 3,3 )( 3,3 )( 3,2 )( 2,2 )
 0.230               606                 ( 2,2 )( 1,4 )( 3,3 )( 3,3 )( 4,1 )( 2,2 )
 0.120               1                   ( 2,2 )( 0,5 )( 3,3 

 Length of FOIS vector: 2612411
 Compute diagonal
  3.588937 seconds (65.31 M allocations: 1.024 GiB, 15.42% gc time)

  Root         E(0)         E(2)
     1 -116.56774078 -116.57609736
     2 -116.56693797 -116.57448761
     3 -116.56535836 -116.57200578
     4 -116.56283256 -116.56842211
     5 -116.55931374 -116.56348191
     6 -116.55472843 -116.55749785
 Length of PT1  vector  2612411 →       85 
 TPSCI Iter 6   Dim: 14010  E(var): -116.56774078 -116.56693797 -116.56535836 -116.56283256 -116.55931374 -116.55472843 

     Selected CI Iteration:    7 epsilon:   0.00100000
 Clip values < -1.0e+00          14010 →  14010
 Add pt vector to current space  14010 →  14095
 In open_matvec_thread2

 Number of jobs:    346
 Number of threads: 12
  0.474097 seconds (3.49 M allocations: 113.540 MiB)
 Now collect thread results
 In open_matvec_thread2

 Number of jobs:    346
 Number of threads: 12
  0.468546 seconds (3.51 M allocations: 114.102 MiB)
 Now collect thread results
 Add spin purif

 0.013               490                 ( 2,2 )( 4,2 )( 3,3 )( 3,3 )( 1,3 )( 2,2 )
 0.013               567                 ( 2,2 )( 1,3 )( 3,3 )( 3,3 )( 4,2 )( 2,2 )
 0.003               23                  ( 2,2 )( 1,5 )( 3,3 )( 3,3 )( 4,0 )( 2,2 )
 0.003               25                  ( 2,2 )( 0,4 )( 3,3 )( 3,3 )( 5,1 )( 2,2 )
 0.013               567                 ( 2,2 )( 3,1 )( 3,3 )( 3,3 )( 2,4 )( 2,2 )
 0.003               23                  ( 2,2 )( 5,1 )( 3,3 )( 3,3 )( 0,4 )( 2,2 )
 0.003               25                  ( 2,2 )( 4,0 )( 3,3 )( 3,3 )( 1,5 )( 2,2 )
 0.001               31                  ( 2,2 )( 1,4 )( 3,3 )( 3,2 )( 4,2 )( 2,2 )
 0.001               32                  ( 2,2 )( 4,1 )( 2,3 )( 3,3 )( 2,4 )( 2,2 )
 0.001               31                  ( 2,2 )( 4,2 )( 3,3 )( 3,2 )( 1,4 )( 2,2 )
 0.001               20                  ( 2,2 )( 4,1 )( 2,3 )( 2,3 )( 3,4 )( 2,2 )
 0.001               32                  ( 2,2 )( 1,4 )( 3,2 )( 3,3 )( 4,2 )

  0.445468 seconds (3.53 M allocations: 114.572 MiB)
 Now collect thread results
 Add spin purifying configs to current space  14211 →  14234
 Build Hamiltonian matrix with dimension: 14234
 79.121262 seconds (1.99 G allocations: 46.049 GiB, 9.93% gc time)
  Root       Energy           S2
     1 -116.56785851   0.00043539
     2 -116.56707390   1.99986745
     3 -116.56545684   5.99979487
     4 -116.56291688  11.99996038
     5 -116.55934887  19.99997037
     6 -116.55472847  29.99999696
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim = 14234  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.140               1                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 0,5 )( 2,2 )
 0.140               609               

 0.013               493                 ( 2,2 )( 4,2 )( 3,3 )( 3,3 )( 1,3 )( 2,2 )
 0.013               568                 ( 2,2 )( 1,3 )( 3,3 )( 3,3 )( 4,2 )( 2,2 )
 0.003               23                  ( 2,2 )( 1,5 )( 3,3 )( 3,3 )( 4,0 )( 2,2 )
 0.003               25                  ( 2,2 )( 0,4 )( 3,3 )( 3,3 )( 5,1 )( 2,2 )
 0.013               568                 ( 2,2 )( 3,1 )( 3,3 )( 3,3 )( 2,4 )( 2,2 )
 0.003               23                  ( 2,2 )( 5,1 )( 3,3 )( 3,3 )( 0,4 )( 2,2 )
 0.003               25                  ( 2,2 )( 4,0 )( 3,3 )( 3,3 )( 1,5 )( 2,2 )
 0.001               31                  ( 2,2 )( 1,4 )( 3,3 )( 3,2 )( 4,2 )( 2,2 )
 0.001               32                  ( 2,2 )( 4,1 )( 2,3 )( 3,3 )( 2,4 )( 2,2 )
 0.001               31                  ( 2,2 )( 4,2 )( 3,3 )( 3,2 )( 1,4 )( 2,2 )
 0.001               20                  ( 2,2 )( 4,1 )( 2,3 )( 2,3 )( 3,4 )( 2,2 )
 0.001               32                  ( 2,2 )( 1,4 )( 3,2 )( 3,3 )( 4,2 )

In [11]:
e2, v2 = FermiCG.tpsci_ci(v1, cluster_ops, clustered_ham,
                            thresh_cipsi=.5e-3, # Threshold for adding to P-space
                            thresh_foi=1e-6,    # Threshold for keeping terms when defining FOIS    
                            thresh_asci=-1,     # Threshold of P-space configs to search from
                            max_iter=10,
                            H0 = "Hcmf",
                            do_s2 = true,
                            thresh_s2 = 1e-7,
                            incremental = true,
                            matvec=3);

 ci_vector     :(14240, 6)
 thresh_cipsi  :0.0005
 thresh_foi    :1.0e-6
 thresh_asci   :-1
 max_iter      :10
 conv_thresh   :0.0001

     Selected CI Iteration:    1 epsilon:   0.00050000
 In open_matvec_thread2

 Number of jobs:    346
 Number of threads: 12
  0.438949 seconds (3.51 M allocations: 112.693 MiB)
 Now collect thread results
 In open_matvec_thread2

 Number of jobs:    346
 Number of threads: 12
  0.443950 seconds (3.51 M allocations: 112.694 MiB)
 Now collect thread results
 Add spin purifying configs to current space  14240 →  14240
 Build Hamiltonian matrix with dimension: 14240
 74.260758 seconds (1.99 G allocations: 46.060 GiB, 10.89% gc time)
  Root       Energy           S2
     1 -116.56786523   0.00008702
     2 -116.56708226   2.00007522
     3 -116.56546228   5.99987701
     4 -116.56292311  11.99997447
     5 -116.55935078  19.99998748
     6 -116.55472847  29.99999696
 --------------------------------------------------
 ---------- Fockspaces in state ------

 0.117               1                   ( 2,2 )( 0,5 )( 3,3 )( 3,3 )( 5,0 )( 2,2 )
 0.013               493                 ( 2,2 )( 2,4 )( 3,3 )( 3,3 )( 3,1 )( 2,2 )
 0.013               493                 ( 2,2 )( 4,2 )( 3,3 )( 3,3 )( 1,3 )( 2,2 )
 0.013               568                 ( 2,2 )( 1,3 )( 3,3 )( 3,3 )( 4,2 )( 2,2 )
 0.003               23                  ( 2,2 )( 1,5 )( 3,3 )( 3,3 )( 4,0 )( 2,2 )
 0.003               25                  ( 2,2 )( 0,4 )( 3,3 )( 3,3 )( 5,1 )( 2,2 )
 0.013               568                 ( 2,2 )( 3,1 )( 3,3 )( 3,3 )( 2,4 )( 2,2 )
 0.003               23                  ( 2,2 )( 5,1 )( 3,3 )( 3,3 )( 0,4 )( 2,2 )
 0.003               25                  ( 2,2 )( 4,0 )( 3,3 )( 3,3 )( 1,5 )( 2,2 )
 0.001               31                  ( 2,2 )( 1,4 )( 3,3 )( 3,2 )( 4,2 )( 2,2 )
 0.001               32                  ( 2,2 )( 4,1 )( 2,3 )( 3,3 )( 2,4 )( 2,2 )
 0.001               31                  ( 2,2 )( 4,2 )( 3,3 )( 3,2 )( 1,4 )

  0.594717 seconds (4.73 M allocations: 156.350 MiB)
 Now collect thread results
 In open_matvec_thread2

 Number of jobs:    816
 Number of threads: 12
  0.673563 seconds (5.33 M allocations: 173.289 MiB)
 Now collect thread results
 Add spin purifying configs to current space  18943 →  22326
 Build Hamiltonian matrix with dimension: 22326
170.141329 seconds (4.02 G allocations: 94.646 GiB, 11.41% gc time)
  Root       Energy           S2
     1 -116.57581203   0.00182218
     2 -116.57491657   1.99874410
     3 -116.57304981   5.99991095
     4 -116.57006425  11.99974249
     5 -116.56574090  20.00011980
     6 -116.55986113  29.99966331
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim = 22326  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             --------- 

 0.001               17                  ( 2,1 )( 5,2 )( 3,3 )( 3,2 )( 0,5 )( 2,2 )
 0.002               24                  ( 2,2 )( 5,2 )( 3,2 )( 3,2 )( 0,5 )( 2,2 )
 0.001               73                  ( 2,2 )( 3,3 )( 3,3 )( 3,2 )( 2,3 )( 2,2 )
 0.002               11                  ( 2,2 )( 1,5 )( 3,3 )( 2,3 )( 5,0 )( 2,2 )
 0.002               11                  ( 2,2 )( 0,5 )( 3,2 )( 3,3 )( 5,1 )( 2,2 )
 0.002               24                  ( 2,2 )( 2,5 )( 2,3 )( 2,3 )( 5,0 )( 2,2 )
 0.001               17                  ( 2,2 )( 5,0 )( 2,3 )( 3,3 )( 2,5 )( 1,2 )
 0.002               11                  ( 2,2 )( 5,0 )( 3,3 )( 2,3 )( 1,5 )( 2,2 )
 0.001               74                  ( 2,2 )( 2,3 )( 3,3 )( 3,2 )( 3,3 )( 2,2 )
 0.001               74                  ( 2,2 )( 3,2 )( 3,3 )( 2,3 )( 3,3 )( 2,2 )
 0.002               11                  ( 2,2 )( 5,1 )( 3,2 )( 3,3 )( 0,5 )( 2,2 )
 0.001               74                  ( 2,2 )( 3,2 )( 2,3 )( 3,3 )( 3,3 )

 Number of jobs:    816
 Number of threads: 12
  0.728472 seconds (5.64 M allocations: 184.316 MiB, 3.15% gc time)
 Now collect thread results
 In open_matvec_thread2

 Number of jobs:    816
 Number of threads: 12
  0.742747 seconds (5.65 M allocations: 184.455 MiB, 2.59% gc time)
 Now collect thread results
 Add spin purifying configs to current space  22326 →  22347
 Length of ASCI vector    22347 →    22347 
 Norm of delta v:
   0.10906108
   0.10906560
   0.10603985
   0.10423717
   0.09907788
   0.08444574
 In open_matvec_thread2

 Number of jobs:    13384
 Number of threads: 12
155.625169 seconds (204.66 M allocations: 10.210 GiB, 0.81% gc time)
 Now collect thread results
157.126752 seconds (215.59 M allocations: 10.736 GiB, 0.80% gc time)
 Length of FOIS vector: 9840693
 Length of FOIS vector: 9818346
 Compute diagonal
 13.228350 seconds (245.46 M allocations: 3.832 GiB, 22.08% gc time)

  Root         E(0)         E(2)
     1 -116.57581203 -116.58075009
     2 -116.57491657 -

...

 0.002               20                  ( 1,2 )( 2,5 )( 3,3 )( 2,3 )( 5,0 )( 2,2 )
 0.003               28                  ( 2,2 )( 0,5 )( 3,2 )( 3,2 )( 5,2 )( 2,2 )
 0.001               86                  ( 2,2 )( 5,1 )( 3,3 )( 2,3 )( 1,4 )( 2,2 )
 0.001               17                  ( 2,2 )( 5,0 )( 3,3 )( 1,3 )( 2,5 )( 2,2 )
 0.001               86                  ( 2,2 )( 1,5 )( 3,3 )( 3,2 )( 4,1 )( 2,2 )
 0.002               24                  ( 2,1 )( 5,2 )( 3,2 )( 3,3 )( 0,5 )( 2,2 )
 0.002               17                  ( 2,2 )( 0,5 )( 3,1 )( 3,3 )( 5,2 )( 2,2 )
 0.002               12                  ( 2,2 )( 1,5 )( 2,3 )( 3,3 )( 5,0 )( 2,2 )
 0.001               90                  ( 2,2 )( 4,1 )( 3,2 )( 3,3 )( 1,5 )( 2,2 )
 0.002               28                  ( 2,2 )( 5,0 )( 2,3 )( 2,3 )( 2,5 )( 2,2 )
 0.002               24                  ( 1,2 )( 2,5 )( 2,3 )( 3,3 )( 5,0 )( 2,2 )
 0.002               17                  ( 2,2 )( 2,5 )( 3,3 )( 1,3 )( 5,0 )

 0.001               28                  ( 2,2 )( 2,5 )( 2,3 )( 2,3 )( 5,0 )( 2,2 )
 0.001               28                  ( 2,2 )( 4,3 )( 3,2 )( 3,2 )( 1,4 )( 2,2 )
 0.001               28                  ( 2,2 )( 3,4 )( 2,3 )( 2,3 )( 4,1 )( 2,2 )
 0.002               86                  ( 2,2 )( 2,4 )( 3,3 )( 2,3 )( 4,1 )( 2,2 )
 0.002               89                  ( 2,2 )( 4,1 )( 3,3 )( 2,3 )( 2,4 )( 2,2 )
 --------------------------------------------------
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim = 28702  
 ----------                root ------:     =     5  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.030               1                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 0,5 )( 2,2 )
 0.271               614                 ( 2,2 )( 4,1 )( 3,3 

 Now collect thread results
 Add spin purifying configs to current space  28702 →  28723
 Length of ASCI vector    28723 →    28723 
 Norm of delta v:
   0.09844364
   0.10199833
   0.03792926
   0.02795098
   0.01673673
   0.01123681
 In open_matvec_thread2

 Number of jobs:    13384
 Number of threads: 12
158.013570 seconds (199.19 M allocations: 7.968 GiB, 0.33% gc time)
 Now collect thread results
159.254489 seconds (210.79 M allocations: 8.458 GiB, 0.32% gc time)
 Length of FOIS vector: 10578321
 Length of FOIS vector: 10549598
 Compute diagonal
 15.503762 seconds (263.74 M allocations: 4.128 GiB, 25.21% gc time)

  Root         E(0)         E(2)
     1 -116.57845548 -116.58282881
     2 -116.57753787 -116.58166431
     3 -116.57561209 -116.57939215
     4 -116.57252168 -116.57574564
     5 -116.56800533 -116.57049092
     6 -116.56177412 -116.56336750
 Length of PT1  vector 10549598 →       17 
 TPSCI Iter 6   Dim: 28723  E(var): -116.57845548 -116.57753787 -116.57561209 -116.572

 0.001               24                  ( 2,1 )( 5,2 )( 3,2 )( 3,3 )( 0,5 )( 2,2 )
 0.001               17                  ( 2,2 )( 0,5 )( 3,1 )( 3,3 )( 5,2 )( 2,2 )
 0.002               12                  ( 2,2 )( 1,5 )( 2,3 )( 3,3 )( 5,0 )( 2,2 )
 0.001               88                  ( 2,2 )( 3,3 )( 3,2 )( 3,3 )( 2,3 )( 2,2 )
 0.002               28                  ( 2,2 )( 5,0 )( 2,3 )( 2,3 )( 2,5 )( 2,2 )
 0.001               24                  ( 1,2 )( 2,5 )( 2,3 )( 3,3 )( 5,0 )( 2,2 )
 0.001               17                  ( 2,2 )( 2,5 )( 3,3 )( 1,3 )( 5,0 )( 2,2 )
 0.001               17                  ( 2,2 )( 5,2 )( 3,1 )( 3,3 )( 0,5 )( 2,2 )
 0.001               17                  ( 2,2 )( 0,5 )( 3,3 )( 3,1 )( 5,2 )( 2,2 )
 0.001               86                  ( 2,2 )( 3,3 )( 3,3 )( 2,3 )( 3,2 )( 2,2 )
 0.001               20                  ( 2,1 )( 5,2 )( 3,3 )( 3,2 )( 0,5 )( 2,2 )
 0.002               28                  ( 2,2 )( 5,2 )( 3,2 )( 3,2 )( 0,5 )

 0.004               1                   ( 2,2 )( 0,5 )( 3,3 )( 3,3 )( 5,0 )( 2,2 )
 0.001               28                  ( 2,2 )( 4,3 )( 2,3 )( 3,2 )( 2,3 )( 2,2 )
 0.001               28                  ( 2,2 )( 4,3 )( 3,2 )( 2,3 )( 2,3 )( 2,2 )
 0.001               18                  ( 2,2 )( 3,4 )( 2,2 )( 3,3 )( 3,2 )( 2,2 )
 0.001               90                  ( 2,2 )( 2,3 )( 3,2 )( 3,3 )( 3,3 )( 2,2 )
 0.001               18                  ( 2,2 )( 4,3 )( 3,3 )( 2,2 )( 2,3 )( 2,2 )
 0.001               88                  ( 2,2 )( 3,3 )( 2,3 )( 3,3 )( 3,2 )( 2,2 )
 0.001               18                  ( 2,2 )( 3,4 )( 3,3 )( 2,2 )( 3,2 )( 2,2 )
 0.001               28                  ( 2,2 )( 3,4 )( 3,2 )( 3,2 )( 2,3 )( 2,2 )
 0.001               28                  ( 2,2 )( 2,3 )( 3,2 )( 2,3 )( 4,3 )( 2,2 )
 0.001               18                  ( 2,2 )( 2,3 )( 2,2 )( 3,3 )( 4,3 )( 2,2 )
 0.001               28                  ( 2,2 )( 3,4 )( 2,3 )( 3,2 )( 3,2 )

In [12]:
pt_e0, pt_v0 = FermiCG.compute_pt2(v0, cluster_ops, clustered_ham, thresh_foi=1e-8, matvec=3)


 .............................do PT2................................
 Norms of input states
   1.00000000
   1.00000000
   1.00000000
   1.00000000
   1.00000000
   1.00000000
 Compute FOIS vector
 In open_matvec_thread2

 Number of jobs:    886
 Number of threads: 12
 12.752572 seconds (7.96 M allocations: 618.576 MiB)
 Now collect thread results
 12.784706 seconds (8.18 M allocations: 651.689 MiB)
 Length of FOIS vector: 3023896
 Length of FOIS vector: 3023364
 Compute diagonal
  4.092626 seconds (75.58 M allocations: 1.150 GiB, 24.09% gc time)
 Compute <0|H0|0>
 Compute <0|H|0>

  Root         E(0)         E(2)
     1 -116.51088649 -116.52433969
     2 -116.51064398 -116.52367936
     3 -116.51015349 -116.52237482
     4 -116.50940640 -116.52046693
     5 -116.50839669 -116.51804531
     6 -116.50713225 -116.51530714


([-0.01345320389995659, -0.013035380715419412, -0.012221337229470948, -0.011060523689941109, -0.009648613989644443, -0.008174890318562023], ClusteredState{Float64,6,6}(Cluster[Cluster(1, [1, 2]), Cluster(2, [3, 4, 5, 6, 7]), Cluster(3, [8, 10, 12]), Cluster(4, [9, 11, 13]), Cluster(5, [14, 15, 16, 17, 18]), Cluster(6, [19, 20])], OrderedCollections.OrderedDict{FockConfig{6},OrderedCollections.OrderedDict{ClusterConfig{6},StaticArrays.MArray{Tuple{6},Float64,1,6}}}(FockConfig{6}(((2, 2), (5, 0), (3, 3), (3, 3), (0, 5), (2, 2))) => OrderedCollections.OrderedDict(),FockConfig{6}(((2, 2), (4, 1), (3, 3), (3, 3), (1, 4), (2, 2))) => OrderedCollections.OrderedDict(ClusterConfig{6}((1, 5, 1, 1, 1, 1)) => [1.833626879992571e-7, 0.0003610775304511798, -0.0006933848506604143, -0.000544007340201556, 0.00018573934701392278, 7.524878492251652e-15],ClusterConfig{6}((1, 6, 1, 1, 1, 1)) => [6.565516963141306e-7, 0.00039261489858892197, -0.0008124512029438894, -0.000702280804752154, 0.00026952547829435

In [13]:
pt_e1, pt_v1 = FermiCG.compute_pt2(v1, cluster_ops, clustered_ham, thresh_foi=1e-8, matvec=3)


 .............................do PT2................................
 Norms of input states
   1.00000000
   1.00000000
   1.00000000
   1.00000000
   1.00000000
   1.00000000
 Compute FOIS vector
 In open_matvec_thread2

 Number of jobs:    6716
 Number of threads: 12
329.712236 seconds (180.08 M allocations: 9.643 GiB, 0.41% gc time)
 Now collect thread results
331.463892 seconds (184.75 M allocations: 10.057 GiB, 0.48% gc time)
 Length of FOIS vector: 15895918
 Length of FOIS vector: 15881678
 Compute diagonal
 20.848656 seconds (397.04 M allocations: 6.148 GiB, 21.85% gc time)
 Compute <0|H0|0>
 Compute <0|H|0>

  Root         E(0)         E(2)
     1 -116.56785851 -116.57361879
     2 -116.56707390 -116.57263011
     3 -116.56545684 -116.57059353
     4 -116.56291688 -116.56740783
     5 -116.55934887 -116.56294770
     6 -116.55472847 -116.55721875


([-0.0057602818153955175, -0.005556202583570306, -0.005136689753042267, -0.0044909537375066155, -0.003598828048013732, -0.0024902802447050454], ClusteredState{Float64,6,6}(Cluster[Cluster(1, [1, 2]), Cluster(2, [3, 4, 5, 6, 7]), Cluster(3, [8, 10, 12]), Cluster(4, [9, 11, 13]), Cluster(5, [14, 15, 16, 17, 18]), Cluster(6, [19, 20])], OrderedCollections.OrderedDict{FockConfig{6},OrderedCollections.OrderedDict{ClusterConfig{6},StaticArrays.MArray{Tuple{6},Float64,1,6}}}(FockConfig{6}(((2, 2), (5, 0), (3, 3), (3, 3), (0, 5), (2, 2))) => OrderedCollections.OrderedDict(),FockConfig{6}(((2, 2), (4, 1), (3, 3), (3, 3), (1, 4), (2, 2))) => OrderedCollections.OrderedDict(ClusterConfig{6}((1, 9, 1, 1, 1, 1)) => [-2.635948182220301e-6, -0.00010609018647512067, 0.0002508815069659214, -0.0002717601378776363, 0.00013951760585489376, -1.1751517601717598e-8],ClusterConfig{6}((1, 21, 1, 1, 1, 1)) => [-5.867265628587986e-7, 5.958126787516323e-5, -0.0001322164534713291, 0.00011951088817867297, -4.0028981

In [14]:
pt_e2, pt_v2 = FermiCG.compute_pt2(v2, cluster_ops, clustered_ham, thresh_foi=1e-8, matvec=3)


 .............................do PT2................................
 Norms of input states
   1.00000000
   1.00000000
   1.00000000
   1.00000000
   1.00000000
   1.00000000
 Compute FOIS vector
 In open_matvec_thread2

 Number of jobs:    13384
 Number of threads: 12
779.968889 seconds (377.92 M allocations: 19.709 GiB, 4.48% gc time)
 Now collect thread results
786.532621 seconds (389.52 M allocations: 20.611 GiB, 4.82% gc time)
 Length of FOIS vector: 26106045
 Length of FOIS vector: 26077291
 Compute diagonal
 36.706288 seconds (651.93 M allocations: 10.148 GiB, 25.07% gc time)
 Compute <0|H0|0>
 Compute <0|H|0>

  Root         E(0)         E(2)
     1 -116.57847235 -116.58268028
     2 -116.57755098 -116.58158750
     3 -116.57562601 -116.57931714
     4 -116.57254134 -116.57570939
     5 -116.56802885 -116.57048679
     6 -116.56180250 -116.56338272


([-0.004207927263456215, -0.004036512879898799, -0.0036911355039154297, -0.0031680497178175563, -0.0024579350302036695, -0.0015802275749103672], ClusteredState{Float64,6,6}(Cluster[Cluster(1, [1, 2]), Cluster(2, [3, 4, 5, 6, 7]), Cluster(3, [8, 10, 12]), Cluster(4, [9, 11, 13]), Cluster(5, [14, 15, 16, 17, 18]), Cluster(6, [19, 20])], OrderedCollections.OrderedDict{FockConfig{6},OrderedCollections.OrderedDict{ClusterConfig{6},StaticArrays.MArray{Tuple{6},Float64,1,6}}}(FockConfig{6}(((2, 2), (5, 0), (3, 3), (3, 3), (0, 5), (2, 2))) => OrderedCollections.OrderedDict(),FockConfig{6}(((2, 2), (4, 1), (3, 3), (3, 3), (1, 4), (2, 2))) => OrderedCollections.OrderedDict(ClusterConfig{6}((1, 9, 1, 1, 1, 1)) => [1.1044707434105228e-6, -9.15640751541008e-5, -0.00022368994531006444, 0.00025306923544431793, -0.000139433101288923, 8.962843704004769e-7],ClusterConfig{6}((1, 21, 1, 1, 1, 1)) => [1.0631217806208568e-6, 0.0001062242696401314, 0.0002512076325488924, -0.00026179906941341496, 0.0001266927

# Now do HOSVD to get better cluster states

In [17]:
rotations = FermiCG.hosvd(v2, cluster_ops)


 --------------------------------------------------------
 Density matrix: Cluster 1

 Compute BRDM
 Hshift = 1.0e-8
 Diagonalize RDM for Cluster  1 in Fock space: (2, 2)
                     :      Population          Energy
   Rotated State    1:      5.90855364    -22.10823808
   ----
   Entanglement entropy:    0.00000000
   Norm:                    5.90855364
 Diagonalize RDM for Cluster  1 in Fock space: (2, 0)
                     :      Population          Energy
   Rotated State    1:      0.00244584    -11.73416682
   ----
   Entanglement entropy:    0.00322341
   Norm:                    0.00244584
 Diagonalize RDM for Cluster  1 in Fock space: (1, 1)
                     :      Population          Energy
   Rotated State    1:      0.00254967    -11.73416682
   Rotated State    2:      0.00013538    -11.43208037
   Rotated State    3:      0.00003199    -11.44545866
   ----
   Entanglement entropy:    0.00365117
   Norm:                    0.00271704
 Diagonalize RDM for C

   Rotated State   22:      0.00002736    -37.42448193
   Rotated State   23:      0.00001929    -37.72527814
   Rotated State   24:      0.00001304    -37.60037887
   Rotated State   25:      0.00001115    -37.65885071
   Rotated State   26:      0.00000259    -37.42607677
   Rotated State   27:      0.00000084    -37.94042648
   Rotated State   28:      0.00000068    -36.98060413
   Rotated State   29:      0.00000060    -37.47407527
   Rotated State   30:      0.00000051    -37.42865938
   Rotated State   31:      0.00000039    -37.42603125
   Rotated State   32:      0.00000028    -37.49959831
   Rotated State   33:      0.00000020    -37.50516378
   Rotated State   34:      0.00000019    -37.38368014
   Rotated State   35:      0.00000017    -37.39012938
   Rotated State   36:      0.00000017    -37.45799686
   Rotated State   37:      0.00000014    -37.48046604
   Rotated State   38:      0.00000013    -37.44907234
   Rotated State   39:      0.00000012    -37.63668456
   Rotated

   Rotated State   55:      0.00000000    -31.50989156
   Rotated State   56:      0.00000000    -31.49401367
   Rotated State   57:      0.00000000    -31.46682153
   Rotated State   58:      0.00000000    -31.46211092
   Rotated State   59:      0.00000000    -31.45112039
   Rotated State   60:      0.00000000    -31.43737106
   Rotated State   61:      0.00000000    -31.38071956
   Rotated State   62:      0.00000000    -31.35441184
   Rotated State   63:      0.00000000    -31.34760135
   Rotated State   64:      0.00000000    -31.25435494
   Rotated State   65:      0.00000000    -31.24430464
   Rotated State   66:      0.00000000    -31.23502121
   Rotated State   67:      0.00000000    -31.22670401
   Rotated State   68:      0.00000000    -31.21628967
   Rotated State   69:      0.00000001    -31.20888146
   Rotated State   70:      0.00000000    -31.18917556
   Rotated State   71:      0.00000000    -31.16682483
   Rotated State   72:      0.00000000    -31.15125888
   Rotated

   Rotated State    3:      0.00000248    -24.21782832
   Rotated State    4:      0.00000147    -24.52999019
   Rotated State    5:      0.00000076    -23.60222482
   Rotated State    6:      0.00000017    -24.56212834
   Rotated State    7:      0.00000000    -24.91345093
   Rotated State    8:      0.00000000    -24.35928652
   Rotated State    9:      0.00000000    -24.27135147
   Rotated State   10:      0.00000000    -24.03042950
   ----
   Entanglement entropy:    0.00004232
   Norm:                    0.00001801
 Diagonalize RDM for Cluster  2 in Fock space: (2, 1)
                     :      Population          Energy
   Rotated State    1:      0.00000596    -24.35043444
   Rotated State    2:      0.00000348    -24.79165051
   Rotated State    3:      0.00000175    -24.22204675
   Rotated State    4:      0.00000103    -24.52482449
   Rotated State    5:      0.00000054    -23.60222482
   Rotated State    6:      0.00000012    -24.56751684
   Rotated State   12:      0.00000

   Entanglement entropy:    0.02314995
   Norm:                    0.02054891
 Diagonalize RDM for Cluster  4 in Fock space: (1, 2)
                     :      Population          Energy
   Rotated State    1:      0.00021471    -21.25120770
   ----
   Entanglement entropy:    0.00036640
   Norm:                    0.00021471
 Diagonalize RDM for Cluster  4 in Fock space: (3, 0)
                     :      Population          Energy
   Rotated State    1:      0.00020342    -21.25120770
   ----
   Entanglement entropy:    0.00034896
   Norm:                    0.00020342
 Diagonalize RDM for Cluster  4 in Fock space: (2, 1)
                     :      Population          Energy
   Rotated State    1:      0.00021474    -21.25120770
   ----
   Entanglement entropy:    0.00036642
   Norm:                    0.00021474
 Diagonalize RDM for Cluster  4 in Fock space: (0, 3)
                     :      Population          Energy
   Rotated State    1:      0.00020352    -21.25120770
   ----


   Rotated State   18:      0.00010974    -37.67138559
   Rotated State   19:      0.00007516    -37.24512504
   Rotated State   20:      0.00003168    -37.35574395
   Rotated State   21:      0.00002836    -37.52033801
   Rotated State   22:      0.00002235    -37.64181664
   Rotated State   23:      0.00001774    -37.56089724
   Rotated State   24:      0.00001144    -37.55083829
   Rotated State   25:      0.00001069    -37.63488437
   Rotated State   26:      0.00000241    -37.41833062
   Rotated State   27:      0.00000078    -37.85285633
   Rotated State   28:      0.00000070    -37.03595146
   Rotated State   29:      0.00000062    -37.44523653
   Rotated State   30:      0.00000047    -37.41160995
   Rotated State   31:      0.00000038    -37.43910866
   Rotated State   32:      0.00000026    -37.43595898
   Rotated State   33:      0.00000020    -37.48613175
   Rotated State   34:      0.00000019    -37.25334482
   Rotated State   35:      0.00000017    -37.70203552
   Rotated

   Rotated State   31:      0.00000011    -31.32100236
   Rotated State   32:      0.00000008    -31.25214658
   Rotated State   33:      0.00000007    -31.14954578
   Rotated State   34:      0.00000006    -31.28270489
   Rotated State   35:      0.00000006    -31.36829264
   Rotated State   36:      0.00000004    -31.23461808
   Rotated State   37:      0.00000004    -31.05491310
   Rotated State   38:      0.00000003    -30.97166120
   Rotated State   39:      0.00000003    -31.06542536
   Rotated State   40:      0.00000002    -31.24205034
   Rotated State   41:      0.00000002    -31.06544402
   Rotated State   42:      0.00000002    -31.21670120
   Rotated State   43:      0.00000001    -30.96567994
   Rotated State   44:      0.00000000    -32.35951582
   Rotated State   45:      0.00000000    -32.06742694
   Rotated State   46:      0.00000000    -32.00128679
   Rotated State   47:      0.00000000    -31.82326652
   Rotated State   48:      0.00000000    -31.81062476
   Rotated

   Rotated State   46:      0.00000001    -43.17075873
   Rotated State   47:      0.00000001    -43.19931699
   Rotated State   48:      0.00000000    -43.15429101
   Rotated State   49:      0.00000000    -42.80882163
   Rotated State   50:      0.00000000    -42.69494400
   ----
   Entanglement entropy:    0.06904320
   Norm:                    0.06643101
 Diagonalize RDM for Cluster  5 in Fock space: (4, 0)
                     :      Population          Energy
   Rotated State    1:      0.02534998    -31.53433652
   Rotated State    2:      0.02087626    -31.07328601
   Rotated State    3:      0.00388663    -30.94601266
   Rotated State    4:      0.00188193    -31.39599112
   Rotated State    5:      0.00079691    -31.48371264
   ----
   Entanglement entropy:    0.05487315
   Norm:                    0.05279172
 Diagonalize RDM for Cluster  5 in Fock space: (5, 1)
                     :      Population          Energy
   Rotated State    1:      0.03110574    -43.39185925
   Ro

   Rotated State    3:      0.00000261    -24.18491806
   Rotated State    4:      0.00000141    -24.52211555
   Rotated State    5:      0.00000077    -23.59464592
   Rotated State    6:      0.00000012    -24.57389817
   Rotated State    7:      0.00000000    -24.92074667
   Rotated State    8:      0.00000000    -24.35667807
   Rotated State    9:      0.00000000    -24.27144568
   Rotated State   10:      0.00000000    -24.02418270
   ----
   Entanglement entropy:    0.00004182
   Norm:                    0.00001778
 Diagonalize RDM for Cluster  5 in Fock space: (2, 1)
                     :      Population          Energy
   Rotated State    1:      0.00000580    -24.34516764
   Rotated State    2:      0.00000342    -24.78439139
   Rotated State    3:      0.00000184    -24.18590140
   Rotated State    4:      0.00000099    -24.52044642
   Rotated State    5:      0.00000055    -23.59464592
   Rotated State    6:      0.00000008    -24.57880879
   Rotated State   12:      0.00000

6-element Array{Any,1}:
 Dict{Tuple,Array{Float64,2}}((1, 2) => [-0.9946411104314099 0.10338791728133338; -0.10338791728133338 -0.9946411104314099],(2, 0) => [1.0],(0, 2) => [1.0],(2, 2) => [1.0],(1, 1) => [0.0 0.0 1.6653345369377348e-16 1.0000000000000002; 1.0 0.0 0.0 0.0; 0.0 -0.6336326255991519 0.7736340839029295 1.1102230246251565e-16; 0.0 0.7736340839029298 0.6336326255991518 -4.440892098500626e-16],(2, 1) => [-0.9945865276048618 0.10391168898109435; -0.10391168898109435 -0.9945865276048617])
 Dict{Tuple,Array{Float64,2}}((3, 5) => [0.0 0.8520498836008495 … -2.127608385032606e-17 -2.981720398215853e-17; 0.0 -4.0120736663884375e-17 … -0.6841609694522041 -0.06052240741427803; … ; 0.0 -1.4491355848370222e-17 … -0.01872238650482891 0.05679184743545734; -0.008304352029632695 -4.1100846085439486e-17 … -2.290066351048839e-17 -1.426455545485216e-17],(4, 3) => [-0.007932434227947418 -0.4093783460405161 … 1.1763118813262415e-16 -1.2504452698875003e-16; -0.9731438033552497 -0.064130762503035

In [18]:
for ci in clusters
    FermiCG.rotate!(cluster_ops[ci.idx], rotations[ci.idx])
    FermiCG.rotate!(cluster_bases[ci.idx], rotations[ci.idx])
    FermiCG.check_basis_orthogonality(cluster_bases[ci.idx])
end


In [19]:
nroots = 6
ci_vector = FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(5,0),(3,3),(3,3),(0,5),(2,2)]), R=nroots);
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(4,1),(3,3),(3,3),(1,4),(2,2)]),R=nroots))
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(3,2),(3,3),(3,3),(2,3),(2,2)]),R=nroots))
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(2,3),(3,3),(3,3),(3,2),(2,2)]),R=nroots))
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(1,4),(3,3),(3,3),(4,1),(2,2)]),R=nroots))
FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, FermiCG.FockConfig([(2,2),(0,5),(3,3),(3,3),(5,0),(2,2)]),R=nroots))


for (fock,configs) in deepcopy(ci_vector.data)
    for ci in clusters
        for cj in clusters
            #a
            new_fock = [fock...]
            new_fock[ci.idx] = (new_fock[ci.idx][1]+1, new_fock[ci.idx][2])
            new_fock[cj.idx] = (new_fock[cj.idx][1]-1, new_fock[cj.idx][2])
            new_fock = FermiCG.FockConfig(new_fock)
            all([f[1]>=0 && f[1]<=length(clusters[ii]) for (ii,f) in enumerate(new_fock)]) || continue
#             FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, new_fock,R=nroots))
        end
    end
    
    for ci in clusters
        for cj in clusters
            #b
            new_fock = [fock...]
            new_fock[ci.idx] = (new_fock[ci.idx][1], new_fock[ci.idx][2]+1)
            new_fock[cj.idx] = (new_fock[cj.idx][1], new_fock[cj.idx][2]-1)
            new_fock = FermiCG.FockConfig(new_fock)
            
            all([f[2]>=0 && f[2]<=length(clusters[ii]) for (ii,f) in enumerate(new_fock)]) || continue
#             FermiCG.add!(ci_vector, FermiCG.ClusteredState(clusters, new_fock,R=nroots))
        end
    end
end
        
display(ci_vector)


 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =     6  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 --------------------------------------------------


In [22]:
e0a, v0a = FermiCG.tpsci_ci(ci_vector, cluster_ops, clustered_ham,
                            thresh_cipsi=5e-3, # Threshold for adding to P-space
                            thresh_foi=1e-5,    # Threshold for keeping terms when defining FOIS    
                            thresh_asci=-1,     # Threshold of P-space configs to search from
                            max_iter=10,
                            do_s2=false,
                            matvec=3);

 ci_vector     :(6, 6)
 thresh_cipsi  :0.005
 thresh_foi    :1.0e-5
 thresh_asci   :-1
 max_iter      :10
 conv_thresh   :0.0001

     Selected CI Iteration:    1 epsilon:   0.00500000
 Build Hamiltonian matrix with dimension:     6
  0.004073 seconds (3.59 k allocations: 94.016 KiB)
  Root       Energy           S2
     1 -116.50708525  29.99285936
     2 -116.49920660  19.99312502
     3 -116.49300691  11.99599099
     4 -116.48843768   5.99992571
     5 -116.48535343   2.00178911
     6 -116.48375941   0.00347594
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =     6  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.004               1                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 0,5 )( 2,2 )
 0.097     

 0.074               1                   ( 2,2 )( 0,5 )( 3,3 )( 3,3 )( 5,0 )( 2,2 )
 0.010               6                   ( 2,2 )( 3,3 )( 3,3 )( 3,3 )( 2,2 )( 2,2 )
 0.005               6                   ( 2,2 )( 2,4 )( 3,3 )( 3,3 )( 3,1 )( 2,2 )
 0.010               6                   ( 2,2 )( 2,2 )( 3,3 )( 3,3 )( 3,3 )( 2,2 )
 0.004               6                   ( 2,2 )( 4,2 )( 3,3 )( 3,3 )( 1,3 )( 2,2 )
 0.005               6                   ( 2,2 )( 1,3 )( 3,3 )( 3,3 )( 4,2 )( 2,2 )
 0.004               6                   ( 2,2 )( 3,1 )( 3,3 )( 3,3 )( 2,4 )( 2,2 )
 --------------------------------------------------
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =   100  
 ----------                root ------:     =     4  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------         

 0.006               6                   ( 2,2 )( 3,1 )( 3,3 )( 3,3 )( 2,4 )( 2,2 )
 0.001               6                   ( 2,2 )( 5,1 )( 3,3 )( 3,3 )( 0,4 )( 2,2 )
 0.001               8                   ( 2,2 )( 4,0 )( 3,3 )( 3,3 )( 1,5 )( 2,2 )
 --------------------------------------------------
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =   169  
 ----------                root ------:     =     5  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.033               1                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 0,5 )( 2,2 )
 0.312               29                  ( 2,2 )( 4,1 )( 3,3 )( 3,3 )( 1,4 )( 2,2 )
 0.142               19                  ( 2,2 )( 3,2 )( 3,3 )( 3,3 )( 2,3 )( 2,2 )
 0.138               20                  ( 2,2 )( 2,3 )( 3,3 

 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.120               1                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 0,5 )( 2,2 )
 0.278               31                  ( 2,2 )( 4,1 )( 3,3 )( 3,3 )( 1,4 )( 2,2 )
 0.086               20                  ( 2,2 )( 3,2 )( 3,3 )( 3,3 )( 2,3 )( 2,2 )
 0.094               21                  ( 2,2 )( 2,3 )( 3,3 )( 3,3 )( 3,2 )( 2,2 )
 0.275               30                  ( 2,2 )( 1,4 )( 3,3 )( 3,3 )( 4,1 )( 2,2 )
 0.117               1                   ( 2,2 )( 0,5 )( 3,3 )( 3,3 )( 5,0 )( 2,2 )
 0.007               7                   ( 2,2 )( 2,4 )( 3,3 )( 3,3 )( 3,1 )( 2,2 )
 0.007               8                   ( 2,2 )( 4,2 )( 3,3 )( 3,3 )( 1,3 )( 2,2 )
 0.007               6                   ( 2,2 )( 1,3 )( 3,3 )( 3,3 )( 4,2 )( 2,2 )
 0.001               7                   ( 2,2 )( 1,5 )( 3,3 )( 3,3 )( 4,0 )( 2,2 )
 0.001               8              

In [25]:
e1a, v1a = FermiCG.tpsci_ci(v0a, cluster_ops, clustered_ham,
                            thresh_cipsi=1e-3, # Threshold for adding to P-space
                            thresh_foi=1e-5,    # Threshold for keeping terms when defining FOIS    
                            thresh_asci=-1,     # Threshold of P-space configs to search from
                            max_iter=10,
                            do_s2=false,
                            thresh_s2=1e-3,
                            incremental=true,
                            matvec=3);

 ci_vector     :(174, 6)
 thresh_cipsi  :0.001
 thresh_foi    :1.0e-5
 thresh_asci   :-1
 max_iter      :10
 conv_thresh   :0.0001

     Selected CI Iteration:    1 epsilon:   0.00100000
 Build Hamiltonian matrix with dimension:   174
  0.020862 seconds (519.70 k allocations: 12.457 MiB)
  Root       Energy           S2
     1 -116.51122227   0.05171077
     2 -116.51100206   2.01529169
     3 -116.51039468   5.95015976
     4 -116.50951210  11.98549172
     5 -116.50842660  19.99836658
     6 -116.50713116  29.99986652
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =   174  
 ----------                root ------:     =     1  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight              # Configs           Fock space(α,β)...  
 -------             ---------           ----------          
 0.192               1                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 0,5 )( 2,2 )
 0.149 

 Length of ASCI vector      174 →      174 
 Norm of delta v:
   1.00000000
   1.00000000
   1.00000000
   1.00000000
   1.00000000
   1.00000000
 In open_matvec_thread2

 Number of jobs:    886
 Number of threads: 12
  1.583867 seconds (2.67 M allocations: 207.530 MiB)
 Now collect thread results
  1.611790 seconds (2.88 M allocations: 238.956 MiB)
 Length of FOIS vector: 547779
 Length of FOIS vector: 547605
 Compute diagonal
  0.802364 seconds (13.69 M allocations: 213.588 MiB)

  Root         E(0)         E(2)
     1 -116.51122227 -116.52488503
     2 -116.51100206 -116.52405633
     3 -116.51039468 -116.52276703
     4 -116.50951210 -116.52069115
     5 -116.50842660 -116.51813025
     6 -116.50713116 -116.51531786
 Length of PT1  vector   547605 →     2331 
 TPSCI Iter 1   Dim: 174    E(var): -116.51122227 -116.51100206 -116.51039468 -116.50951210 -116.50842660 -116.50713116 

     Selected CI Iteration:    2 epsilon:   0.00100000
 Clip values < -1.0e+00            174 →    174
 

 Length of ASCI vector     2505 →     2505 
 Norm of delta v:
   1.00231915
   0.86572027
   0.80142100
   0.45072910
   0.33111546
   0.21950149
 In open_matvec_thread2

 Number of jobs:    6674
 Number of threads: 12
 22.885439 seconds (27.62 M allocations: 1.595 GiB, 8.31% gc time)
 Now collect thread results
 24.396615 seconds (31.45 M allocations: 1.769 GiB, 12.34% gc time)
 Length of FOIS vector: 2051217
 Length of FOIS vector: 2048712
 Compute diagonal
  3.823644 seconds (51.22 M allocations: 814.460 MiB, 43.32% gc time)

  Root         E(0)         E(2)
     1 -116.55824335 -116.56379654
     2 -116.55802522 -116.56500672
     3 -116.54433716 -116.55448079
     4 -116.54315409 -116.55095823
     5 -116.54010440 -116.54600367
     6 -116.53715258 -116.54169345
 Length of PT1  vector  2048712 →     7043 
 TPSCI Iter 2   Dim: 2505   E(var): -116.55824335 -116.55802522 -116.54433716 -116.54315409 -116.54010440 -116.53715258 

     Selected CI Iteration:    3 epsilon:   0.00100000
 

 0.001               11                  ( 2,2 )( 1,4 )( 3,2 )( 3,2 )( 4,3 )( 2,2 )
 0.001               7                   ( 2,2 )( 4,2 )( 3,2 )( 3,3 )( 1,4 )( 2,2 )
 0.001               12                  ( 2,2 )( 4,3 )( 3,2 )( 3,2 )( 1,4 )( 2,2 )
 0.001               12                  ( 2,2 )( 3,4 )( 2,3 )( 2,3 )( 4,1 )( 2,2 )
 0.001               8                   ( 2,2 )( 2,4 )( 3,3 )( 2,3 )( 4,1 )( 2,2 )
 0.001               10                  ( 1,2 )( 3,4 )( 2,3 )( 3,3 )( 4,1 )( 2,2 )
 0.001               10                  ( 2,2 )( 4,1 )( 3,3 )( 2,3 )( 2,4 )( 2,2 )
 0.001               10                  ( 1,2 )( 3,4 )( 3,3 )( 2,3 )( 4,1 )( 2,2 )
 --------------------------------------------------
 --------------------------------------------------
 ---------- Fockspaces in state ------: Dim =  9548  
 ----------                root ------:     =     5  
 --------------------------------------------------
 Printing contributions greater than: 0.001000
 Weight          

 0.002               41                  ( 2,2 )( 2,5 )( 2,3 )( 3,3 )( 4,0 )( 2,2 )
 0.005               8                   ( 2,2 )( 1,5 )( 2,3 )( 3,3 )( 5,0 )( 2,2 )
 0.002               40                  ( 2,2 )( 0,4 )( 3,3 )( 3,2 )( 5,2 )( 2,2 )
 0.001               17                  ( 2,2 )( 1,5 )( 2,2 )( 3,3 )( 5,1 )( 2,2 )
 0.004               19                  ( 1,2 )( 2,5 )( 2,3 )( 3,3 )( 5,0 )( 2,2 )
 0.004               17                  ( 2,2 )( 2,5 )( 3,3 )( 1,3 )( 5,0 )( 2,2 )
 0.001               15                  ( 2,2 )( 1,5 )( 3,3 )( 2,2 )( 5,1 )( 2,2 )
 0.004               14                  ( 2,2 )( 0,5 )( 3,3 )( 3,1 )( 5,2 )( 2,2 )
 0.003               27                  ( 2,2 )( 1,4 )( 2,3 )( 3,3 )( 5,1 )( 2,2 )
 0.002               41                  ( 2,2 )( 2,5 )( 3,3 )( 2,3 )( 4,0 )( 2,2 )
 0.005               9                   ( 2,2 )( 1,5 )( 3,3 )( 2,3 )( 5,0 )( 2,2 )
 0.006               8                   ( 2,2 )( 0,5 )( 3,2 )( 3,3 )( 5,1 )

...

 0.001               4                   ( 2,2 )( 5,0 )( 3,3 )( 3,3 )( 2,5 )( 0,2 )
 0.001               18                  ( 2,2 )( 5,1 )( 2,2 )( 3,3 )( 1,5 )( 2,2 )
 0.001               18                  ( 2,2 )( 4,0 )( 3,3 )( 3,3 )( 2,5 )( 1,2 )
 0.002               41                  ( 2,2 )( 4,0 )( 3,3 )( 2,3 )( 2,5 )( 2,2 )
 0.006               27                  ( 2,2 )( 5,0 )( 2,3 )( 2,3 )( 2,5 )( 2,2 )
 0.004               15                  ( 2,2 )( 5,2 )( 3,1 )( 3,3 )( 0,5 )( 2,2 )
 0.004               21                  ( 2,1 )( 5,2 )( 3,3 )( 3,2 )( 0,5 )( 2,2 )
 0.006               32                  ( 2,2 )( 5,2 )( 3,2 )( 3,2 )( 0,5 )( 2,2 )
 0.001               18                  ( 2,1 )( 5,2 )( 3,3 )( 3,3 )( 0,4 )( 2,2 )
 0.004               21                  ( 2,2 )( 5,0 )( 2,3 )( 3,3 )( 2,5 )( 1,2 )
 0.002               44                  ( 2,2 )( 5,2 )( 3,3 )( 3,2 )( 0,4 )( 2,2 )
 0.005               8                   ( 2,2 )( 5,0 )( 3,3 )( 2,3 )( 1,5 )

 0.001               6                   ( 2,2 )( 4,3 )( 3,3 )( 2,2 )( 2,3 )( 2,2 )
 0.001               6                   ( 2,2 )( 3,4 )( 3,3 )( 2,2 )( 3,2 )( 2,2 )
 0.001               8                   ( 2,2 )( 2,3 )( 2,2 )( 3,3 )( 4,3 )( 2,2 )
 0.001               8                   ( 2,2 )( 3,2 )( 3,3 )( 2,2 )( 3,4 )( 2,2 )
 0.001               8                   ( 2,2 )( 2,3 )( 3,3 )( 2,2 )( 4,3 )( 2,2 )
 0.001               8                   ( 2,2 )( 3,2 )( 2,2 )( 3,3 )( 3,4 )( 2,2 )
 0.001               9                   ( 2,2 )( 4,3 )( 2,2 )( 3,3 )( 2,3 )( 2,2 )
 --------------------------------------------------
 Length of ASCI vector    17847 →    17847 
 Norm of delta v:
   1.13421929
   1.13422222
   0.01786623
   0.01785236
   0.01915961
   0.01070375
 In open_matvec_thread2

 Number of jobs:    7554
 Number of threads: 12
 36.752483 seconds (106.90 M allocations: 3.790 GiB, 14.63% gc time)
 Now collect thread results
 37.333093 seconds (112.00 M allocations: 4.

 0.001               10                  ( 2,1 )( 4,3 )( 3,3 )( 3,2 )( 1,4 )( 2,2 )
 0.001               11                  ( 2,1 )( 4,3 )( 3,2 )( 3,3 )( 1,4 )( 2,2 )
 0.001               19                  ( 2,2 )( 2,3 )( 3,3 )( 2,3 )( 4,2 )( 2,2 )
 0.002               11                  ( 2,2 )( 4,2 )( 3,3 )( 3,2 )( 1,4 )( 2,2 )
 0.002               16                  ( 2,2 )( 4,1 )( 2,3 )( 2,3 )( 3,4 )( 2,2 )
 0.001               9                   ( 2,2 )( 4,1 )( 3,3 )( 1,3 )( 3,4 )( 2,2 )
 0.002               13                  ( 2,2 )( 1,4 )( 3,2 )( 3,3 )( 4,2 )( 2,2 )
 0.001               22                  ( 2,2 )( 2,4 )( 3,3 )( 3,2 )( 3,2 )( 2,2 )
 0.001               19                  ( 2,2 )( 4,2 )( 3,3 )( 2,3 )( 2,3 )( 2,2 )
 0.001               19                  ( 2,2 )( 2,3 )( 2,3 )( 3,3 )( 4,2 )( 2,2 )
 0.001               9                   ( 2,2 )( 4,1 )( 1,3 )( 3,3 )( 3,4 )( 2,2 )
 0.002               12                  ( 2,2 )( 2,4 )( 2,3 )( 3,3 )( 4,1 )

 0.005               21                  ( 2,1 )( 5,2 )( 3,3 )( 3,2 )( 0,5 )( 2,2 )
 0.006               32                  ( 2,2 )( 5,2 )( 3,2 )( 3,2 )( 0,5 )( 2,2 )
 0.001               18                  ( 2,1 )( 5,2 )( 3,3 )( 3,3 )( 0,4 )( 2,2 )
 0.005               21                  ( 2,2 )( 5,0 )( 2,3 )( 3,3 )( 2,5 )( 1,2 )
 0.002               44                  ( 2,2 )( 5,2 )( 3,3 )( 3,2 )( 0,4 )( 2,2 )
 0.006               8                   ( 2,2 )( 5,0 )( 3,3 )( 2,3 )( 1,5 )( 2,2 )
 0.006               8                   ( 2,2 )( 5,1 )( 3,2 )( 3,3 )( 0,5 )( 2,2 )
 0.006               8                   ( 2,2 )( 5,1 )( 3,3 )( 3,2 )( 0,5 )( 2,2 )
 0.006               8                   ( 2,2 )( 5,0 )( 2,3 )( 3,3 )( 1,5 )( 2,2 )
 0.004               17                  ( 2,2 )( 5,2 )( 3,3 )( 3,1 )( 0,5 )( 2,2 )
 0.003               27                  ( 2,2 )( 5,1 )( 2,3 )( 3,3 )( 1,4 )( 2,2 )
 0.004               15                  ( 2,2 )( 5,0 )( 1,3 )( 3,3 )( 2,5 )

 0.001               8                   ( 2,2 )( 2,3 )( 3,3 )( 2,2 )( 4,3 )( 2,2 )
 0.001               8                   ( 2,2 )( 3,2 )( 2,2 )( 3,3 )( 3,4 )( 2,2 )
 0.001               9                   ( 2,2 )( 4,3 )( 2,2 )( 3,3 )( 2,3 )( 2,2 )
 --------------------------------------------------
 Length of ASCI vector    18202 →    18202 
 Norm of delta v:
   0.07135919
   0.07132010
   0.00568476
   0.00661791
   0.00551858
   0.00255609
 In open_matvec_thread2

 Number of jobs:    7554
 Number of threads: 12
 17.852068 seconds (100.67 M allocations: 3.180 GiB, 12.61% gc time)
 Now collect thread results
 18.318447 seconds (105.81 M allocations: 3.371 GiB, 12.69% gc time)
 Length of FOIS vector: 2814579
 Length of FOIS vector: 2796377
 Compute diagonal
  6.016528 seconds (69.91 M allocations: 1.092 GiB, 51.79% gc time)

  Root         E(0)         E(2)
     1 -116.57346142 -116.57885684
     2 -116.57344622 -116.57885712
     3 -116.55795082 -116.56651390
     4 -116.55542650 -